# Семинар 1: _Индекс_

## Intro

### Чтение файла 
- конструкция __with open__ (recommended)
- конструкция __open + close__

```python
fpath = "fpath.txt"

# одним массивом  
with open(fpath, "r") as f:  
    text = f.read() 

# по строкам, в конце каждой строки \n  
with open(fpath, "r") as f:   
    text = f.readlines() 

# по строкам, без \n   
with open(fpath, "r") as f:   
    text = f.read().splitlines() 
    
# not recommended  
file = open(txt_fpath, "r")  
text = file.read()    
file.close() 
```

### Работа с файлами и папками
#### os.path  
Путь до файла:

```python
import os

# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath("fpath.txt"))

# возвращает имя файла / папки по полному пути до него
print(os.path.basename("/your/path/to/folder/with/fpath.txt"))

# проверить существование директории - True / False
print(os.path.exists("your/path/to/any/folder/"))
```

#### os.listdir  
* Возвращает список файлов в данной директории:

```python
main_dir = "/your/path/to/folder/with/folders/"
os.listdir(main_dir)
```

* Сделаем пути абсолютными, чтобы наш код не зависел от того, где лежит этот файл:
```python
[main_dir + fpath for fpath in os.listdir(main_dir)]
```

* Не забывайте исключать системные директории, такие как `.DS_Store`
```python
[main_dir + fpath for fpath in os.listdir(main_dir) if not ".DS_Store" in fpath]
```

#### os.walk
`root` — начальная директория  
`dirs` — список поддиректорий (папок)   
`files` — список файлов в этих поддиректориях  

```python
main_dir = "/your/path/to/folder/with/folders/"

for root, dirs, files in os.walk(main_dir):
    for name in files:
        print(os.path.join(root, name))
```

> __os.walk__ возвращает генератор. Это значит, что получить его элементы можно, только проитерировавшись по нему, но его легко можно превратить в `list` и увидеть все его значения

```python
list(os.walk(main_dir))
```

##  Обратный индекс 

Сам по себе обратный индекс не может осуществлять поиск, для этого необходимо добавить к нему определенную метрику. Это не совсем очевидная задача, поэтому немного отложим ее. А сейчас посмотрим, что полезного можно вытащить из индекса.    
По сути, индекс — это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:

1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе. Так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. какой документ является самым большим / маленьким (очень изощренный способ, когда есть `len`)

### 1.1 &emsp; __Задача__:  получить обратный индекс для коллекции документов.

Перед этим постройте матрицу терм-документ и сделайте функцию булева поиска, которая по запросу будет возвращать 5 релевантных документов.   
В качестве коллекции возьмите сценарий сезонов сериала Друзья. Одна серия — один документ.

Скачать корпус можно [тут](https://yadi.sk/d/k_M7n63A3adGSz).

Этапы:   

1. получить коллекцию документов
2. для каждого файла коллекции сделать необходимую на ваш взгляд предобработку
3. получить матрицу терм-документ, написать функцию поиска по ней
4. получить обратный индекс в виде словаря, где ключ — нормализованное слово, значение — список файлов, в которых это слово встречается
5. вывести кусочек индекса в виде таблицы 
6. сделать анализ обратного индекса. Это задание принимается в виде кода и ответов на вопросы

Напоминание:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер:
```python 
for i, element in enumerate(your_list): 
    ... 
```    
Иногда для получения элемента делают так — `your_list[i]`, старайтесь этого избегать.

### Подгружаем файлы

In [1]:
import os

In [2]:
main_dir = "./Friends"
files_list = []

### пройдитесь по всем папкам коллекции и соберите все пути .txt файлов
for root, dirs, files in os.walk(main_dir):
    for season_dir in dirs:
        half_path = root + "/" + season_dir
        files_list += [half_path + "/" + fpath for fpath in os.listdir(half_path) if not ".DS_Store" in fpath]

In [3]:
### _check : в коллекции должно быть 165 файлов
len(files_list)

165

### Предобработка

In [4]:
import re
import string

from pymystem3 import Mystem

In [5]:
reg_punct = re.compile("[«–»—!\$%&'()*+,./:;<=>?@^_`{|}~']*")
reg_num = re.compile("[0-9]+")
reg_latin = re.compile("[a-z]+")
reg_episode = re.compile("[0-9]x[0123456789-]+")

mystem = Mystem()

In [6]:
def preprocessing(text):
    text = text.lower()
    
    text = text.replace("\n", " ")
    text = text.replace("\ufeff", "")
    
    text = reg_punct.sub("", text)
    text = reg_num.sub("", text)
    text = reg_latin.sub("", text)
    return text

In [7]:
texts_ready = []
lemmas_ready = []
episodes = []
reg_word = re.compile("[А-ЯЁа-яё]+(?:(?:-[А-ЯЁа-яё]+)*)?")
trash = set([" ", "-", "\n", ""])

for script_path in files_list:
    with open(script_path, "r", encoding="utf-8") as f:
        text_raw = f.read()
    
    text_processed = preprocessing(text_raw)
    tokens = re.findall(reg_word, text_processed)
    
    lemmas = []
    for token in tokens:
        lemma = mystem.lemmatize(token)
        lemmas += [l for l in lemma[:-1] if l not in trash]
    lemmas_ready.append(lemmas)
    text_lemmatized = " ".join(lemmas)
    texts_ready.append(text_lemmatized)
    
    episode = re.search(reg_episode, script_path).group(0)
    episodes.append(episode)

### Матрица терм-документ

In [8]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
cv = CountVectorizer()

In [10]:
term_doc_matrix = pd.DataFrame(cv.fit_transform(texts_ready).A, 
                               columns=cv.get_feature_names(), 
                               index=None)
# столбец episodes будет индексом, чтобы мы понимали, о каком эпизоде речь
term_doc_matrix["episode"] = episodes
term_doc_matrix = term_doc_matrix.set_index(term_doc_matrix["episode"])
del term_doc_matrix["episode"]
term_doc_matrix.head()

,аа,ааа,ааааа,ааааааа,аааааау,ааааах,аарон,аббатство,абонемент,абрикос,...,ярмарка,ярость,ясмин,ясно,ясность,ясный,яхта,ящерица,ящик,ящичек
episode,,,,,,,,,,,,,,,,,,,,,
2x08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2x20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2x24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2x16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2x07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


#### Side note: булев поиск

In [ ]:
# это простой поиск по одному токену
def simple_boolean_search(query) -> list:
    """
    Produces a Boolean search according with the term-document matrix
    :return: list of first 5 relevant documents
    """
    try:
        res = term_doc_matrix.index[term_doc_matrix[query] != 0].tolist()
        return res
    except:
        return []

In [ ]:
# запросы 
input_text = [
    "Моника & Фиби & Рэйчел & Чендлер & Джои & Росс",
    "(Моника ИЛИ Фиби) & Рэйчел & (Чендлер ИЛИ Джои) & Росс", 
    "(НЕ Моника) & Фиби & Рэйчел & Чендлер & Джои & (НЕ Росс)"
]

### Обратный индекс

Совет для построения обратного индекса: 
> В качестве словаря используйте `defaultdict` из модуля `collections`   
Так можно избежать конструкции 
```python 
dict.setdefault(key, default=None) 
```

In [11]:
from collections import Counter, defaultdict

In [12]:
def inverted_index(texts_ready, episodes) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    inv_index = defaultdict(dict)
    for i, text in enumerate(texts_ready):
        episode = episodes[i]
        count = Counter(mystem.lemmatize(text))
        for data in count.most_common():
            lemma, res = data
            if lemma not in trash:
                inv_index[lemma][episode] = res
    return inv_index

In [13]:
inv_index = inverted_index(texts_ready, episodes)

In [14]:
sorted(inv_index)[:5]

['а', 'а-ля', 'аа', 'ааа', 'ааааа']

#### Аналитика
С помощью обратного индекса произведите следующую аналитику:  

1) общая аналитика
- какое слово является самым частотным?
- какое самым редким?
- какой набор слов есть во всех документах коллекции?

2) частота встречаемости имен главных героев в каждом сезоне      
- какой сезон был самым популярным у Чендлера? у Моники?   
- кто из главных героев статистически самый популярный? 


_Самое частотное:_

In [15]:
max_freq = 0
max_freq_word = ""
for lemma in inv_index:
    freq = sum(inv_index[lemma].values())
    if freq > max_freq:
        max_freq = freq
        max_freq_word = lemma

In [16]:
max_freq_word

'я'

_Самое редкое:_

In [17]:
min_freq = max_freq
min_freq_word = ""

In [18]:
for lemma in inv_index:
    freq = sum(inv_index[lemma].values())
    if freq < max_freq:
        min_freq = freq
        min_freq_word = lemma

In [19]:
min_freq_word

'пожа'

_Есть везде:_

In [20]:
present_everywhere = []
for lemma in inv_index:
    if len(inv_index[lemma]) == 165:
        present_everywhere.append(lemma)

In [21]:
", ".join(present_everywhere)

''

_Самые популярные сезоны:_

In [22]:
def most_popular_season(character):
    seasons = {str(num): 0 for num in range(1,8)}
    character = character.lower().strip(" \n")
    character = mystem.lemmatize(character)[0]
    for episode in inv_index[character]:
        season = episode[0]
        seasons[season] += inv_index[character][episode]
    return max(seasons, key=seasons.get)

In [23]:
most_popular_season("Моника")

'7'

In [24]:
most_popular_season("Чендлер")

'6'

_Самый популярный персонаж:_

In [25]:
max_char_freq = 0
most_popular_char = ""
for character in ["Рейчел", "Моника", "Фиби", "Джоуи", "Чендлер", "Росс"]:
    character = character.lower().strip(" \n")
    character = mystem.lemmatize(character)[0]
    char_freq = sum(inv_index[character].values())
    if char_freq > max_char_freq:
        max_char_freq = char_freq
        most_popular_char = character
print(most_popular_char)

росс


## Функция ранжирования Okapi BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{(k_1+1)*f(q_i,D)}{f(q_i,D)+k_1(1-b+b\frac{|D|}{avgdl})} $$ 
где   
>$f(q_i,D)$ - частота слова $q_i$ в документе $D$ (TF)       
$|D|$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k_1$ и $b$ — свободные коэффициенты, обычно их выбирают как $k_1$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ есть обратная документная частота (IDF) слова $q_i$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [ ]:
from math import log
from statistics import mean

In [ ]:
k1 = 2.0
b = 0.75

N = 165
avgdl = mean([len(doc) for doc in lemmas_ready])

In [ ]:
def compute_idf(query):
    n_q = len(inv_index[query])
    frac = (N - n_q + 0.5)/(n_q + 0.5)
    idf = log(frac)
    return idf

In [ ]:
def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    return 

0

### __Задача__:    
напишите функцию, которая сортирует поисковую выдачу для любого входящего запроса согласно метрике *Okapi BM25*.    
Выведите 10 первых результатов и их скор по запросу **рождественские каникулы**. 

In [ ]:
def compute_sim() -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    return 


def get_search_result() -> list:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    return 